```bash
export PATH=${SPARK_HOME}/bin:${PATH}
export SPARK_LOCAL_IP=localhost
export PYSPARK_DRIVER_PYTHON=jupyter
export PYSPARK_DRIVER_PYTHON_OPTS='lab --ip=0.0.0.0 --allow-root --no-browser --NotebookApp.token=""'
export PACKAGES="com.esri:filegdb:0.12.5"
export PACKAGES="${PACKAGES},com.esri:spark-functions:0.10"
pyspark\
  --master local[*]\
  --num-executors 1\
  --driver-memory 30G\
  --executor-memory 30G\
  --conf spark.ui.enabled=false\
  --packages ${PACKAGES}\
  --exclude-packages org.scala-lang:scala-reflect
```

In [ ]:
import os

In [ ]:
count = 0
while count < 2:
    count += 1
    try:
        spark._jvm.com.esri.spark.Functions.registerHaversine()
        sql("select round(haversine(0., 0., 45., 45.),1) as h").show()
    except:
        pass

In [ ]:
gdb = spark._jvm.com.esri.gdb.FileGDB

In [ ]:
gdb_path = os.path.join("data", "Miami.gdb")
for tab in gdb.listTables(gdb_path):
    print(tab)

In [ ]:
df = spark.read \
    .format('com.esri.gdb') \
    .options(path=gdb_path, name='Broadcast') \
    .load()

In [ ]:
df.printSchema()

In [ ]:
df.count()

In [ ]:
df.registerTempTable("bc")

In [ ]:
sql("""
select mmsi,dt1,x1,y1,dt2,x2,y2,haversine(x1,y1,x2,y2) as meters from (
select MMSI mmsi,
BaseDateTime as dt1,
Shape.x as x1,
Shape.y as y1,
lead(BaseDateTime) over (partition by MMSI order by BaseDateTime) as dt2,
lead(Shape.x) over (partition by MMSI order by BaseDateTime) as x2,
lead(Shape.y) over (partition by MMSI order by BaseDateTime) as y2
from bc
)
where dt1 < dt2
having meters > 500.0
""").show(32, truncate=True)